In [24]:
import warnings
import pandas as pd 
import requests
from difflib import SequenceMatcher as SM

import constants as const

In [10]:
warnings.filterwarnings('ignore')

In [18]:
diseases = pd.read_csv('non-icd-diseases.csv')
diseases = diseases[['name']]

In [47]:
diseases.shape

(1599, 1)

In [62]:
def get_icd_disease_equivalent(row):
    l_match = []
    match_dict = {}
    disease_name = row['name'].lower().replace(' ', '-').strip()
    disease_name = disease_name.replace('?', '')
    url = const.ICD_URL.format(disease_name)
    response = requests.get(url)
    matches = response.json()[3]
    if matches:
        for match in matches:
            similarity_score = round(SM(None, disease_name, match[1].lower()).ratio(), 2)
            match_dict[match[1]] = similarity_score
    
    if match_dict:
        max_match = max(match_dict, key=match_dict.get)
        print(disease_name)
        row['max-match'] = max_match
        row['other-matches']  = match_dict
        row['max-score'] = match_dict[max_match]
        print('-----------------')
    return row
    


In [58]:
diseases_100 = diseases.head(100)

In [63]:
diseases = diseases.apply(get_icd_disease_equivalent, axis=1)

sepsis
-----------------
scabies
-----------------
chondritis
-----------------
diabetes-foot-ulcer
-----------------
ankle-osteoarthritis
-----------------
throat-pain
-----------------
surgical-site-infection
-----------------
tendinitis
-----------------
hypertensive-heart-disease
-----------------
shoulder-arthritis
-----------------
-hypothyrodism
-----------------
myositis
-----------------
left-otitis-externa
-----------------
aspiration-pneumonitis
-----------------
adverse-drug-reaction
-----------------
skin-laceration
-----------------
none
-----------------
lumbago
-----------------
-severe-pre-eclampsia,--latent-phase-of-labour.
-----------------
-absence-seizures
-----------------
left-otitis-media
-----------------
malaria-+-sepsis
-----------------
diarrhea
-----------------
prematurity
-----------------
-haemoglobinopathy
-----------------
-plasmodiasis
-----------------
first-trimester-pregnancy
-----------------
hyperemesis-gravidarum
-----------------
c/o:-headache


In [60]:
diseases_100.head()

,max-match,max-score,name,other-matches
0,NaN,NaN,DELAYED WOUND HEALING,NaN
1,NaN,NaN,?malaria in pregnancy,NaN
2,Anthrax sepsis,0.6,?sepsis,"{'Salmonella sepsis': 0.52, 'Anthrax sepsis': ..."
3,NaN,NaN,bacterial vaginosis,NaN
4,NaN,NaN,secondary hypertension with poor compliance,NaN


In [22]:
const.ICD_URL

'https://clinicaltables.nlm.nih.gov/api/icd10cm/v3/search?sf=code,name&terms={}'

In [64]:
diseases.head(10)

,max-match,max-score,name,other-matches
0,NaN,NaN,DELAYED WOUND HEALING,NaN
1,NaN,NaN,?malaria in pregnancy,NaN
2,Anthrax sepsis,0.6,?sepsis,"{'Salmonella sepsis': 0.52, 'Anthrax sepsis': ..."
3,NaN,NaN,bacterial vaginosis,NaN
4,NaN,NaN,secondary hypertension with poor compliance,NaN
5,NaN,NaN,sensoneural deafness,NaN
6,NaN,NaN,STILL HAVING BACK PAIN,NaN
7,NaN,NaN,?osteoarthritis of the knee and shoulder,NaN
8,NaN,NaN,hyernatriemia ?cause,NaN
9,NaN,NaN,patient still vomiting,NaN


In [66]:
no_non_diseases = diseases[diseases['max-score'].notnull()]

In [70]:
no_non_diseases.head()

,name,max-match,max-score,other-matches
2,?sepsis,Anthrax sepsis,0.60,"{'Salmonella sepsis': 0.52, 'Anthrax sepsis': ..."
10,?SCABIES,Scabies,1.00,{'Scabies': 1.0}
20,CHONDRITIS,Chondritis of left external ear,0.49,"{'Chondritis of right external ear': 0.48, 'Ch..."
33,Diabetes Foot Ulcer,Type 1 diabetes mellitus with foot ulcer,0.58,{'Type 1 diabetes mellitus with foot ulcer': 0...
35,ankle osteoarthritis,"Primary osteoarthritis, left ankle and foot",0.48,"{'Primary osteoarthritis, right ankle and foot..."


In [69]:
no_non_diseases = no_non_diseases[['name', 'max-match', 'max-score', 'other-matches']]

In [71]:
no_non_diseases.to_csv('non-icd-diseases-icd-equivalents.csv', index=False)